Implementing Batch and Layer Normalisation in NN using Pytorch



In [ ]:
import torch
import torch.nn.functional as F

# Input data (batch size=4, number of previous layer neurons=2)
before_values = [[4, 2], [3, 1], [4, 5], [6, 2]]

# Convert data to PyTorch tensor
X = torch.tensor(before_values, dtype=torch.float32)
print(f'Original Values: {X}')
print(f'X shape: {X.shape}')

# Apply batch normalisation
def batch_normalisation(X):
  # Calculate mean and variance for each neuron's output
  mean = torch.mean(X, dim=0, keepdim=True) #Batch dimension
  print(f'mean: {mean}')
  std = torch.std(X, dim=0, keepdim=True)
  print(f'standard deviation: {std}')

  # Apply batch normalisation
  gamma = torch.ones_like(mean)  # scale parameter
  beta = torch.zeros_like(mean)  # drift parameter
  epsilon = 1e-8  # small constant to avoid division by zero
  normalised_values = (X - mean) / (std + epsilon)
  return normalised_values

def layer_normalisation(X):
  # Calculate mean and variance for each datapoint
  mean = torch.mean(X, dim=1, keepdim=True) #Feature Dimension
  print(f'mean: {mean}')
  std = torch.std(X, dim=1, keepdim=True)
  print(f'standard deviation: {std}')

  # Apply layer normalisation
  gamma = torch.ones_like(mean)  # scale parameter
  beta = torch.zeros_like(mean)  # drift parameter
  epsilon = 1e-8  # small constant to avoid division by zero
  normalised_values = gamma * (X - mean) / (std + epsilon) + beta
  return normalised_values

print(f'\n----BATCH NORMALISATION----')
batch_normalised_values = batch_normalisation(X)
print(f'Output after applying Batch Normalisation: {batch_normalised_values}')

print(f'\n----LAYER NORMALISATION----')
layer_normalised_values = layer_normalisation(X)
print(f'Output after applying Layer Normalisation: {layer_normalised_values}')

Original Values: tensor([[4., 2.],
        [3., 1.],
        [4., 5.],
        [6., 2.]])
X shape: torch.Size([4, 2])

----BATCH NORMALISATION----
mean: tensor([[4.2500, 2.5000]])
standard deviation: tensor([[1.2583, 1.7321]])
Output after applying Batch Normalisation: tensor([[-0.1987, -0.2887],
        [-0.9934, -0.8660],
        [-0.1987,  1.4434],
        [ 1.3908, -0.2887]])

----LAYER NORMALISATION----
mean: tensor([[3.0000],
        [2.0000],
        [4.5000],
        [4.0000]])
standard deviation: tensor([[1.4142],
        [1.4142],
        [0.7071],
        [2.8284]])
Output after applying Layer Normalisation: tensor([[ 0.7071, -0.7071],
        [ 0.7071, -0.7071],
        [-0.7071,  0.7071],
        [ 0.7071, -0.7071]])


In [ ]:
import torch.nn as nn
import torch

class Net(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(input_dim, hidden_dim)
    self.batch_norm1 = nn.BatchNorm1d(hidden_dim)
    self.relu = nn.ReLU()

    self.fc2 = nn.Linear(hidden_dim, hidden_dim)
    self.layer_norm2 = nn.LayerNorm(hidden_dim)
    self.relu2 = nn.ReLU()

    self.fc3 = nn.Linear(hidden_dim, output_dim)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    x = self.fc1(x)
    x = self.batch_norm1(x)  # Apply batch normalization
    x = self.relu(x)

    x = self.fc2(x)
    x = self.layer_norm2(x)  # Apply layer normalization
    x = self.relu2(x)

    x = self.fc3(x)
    x = self.sigmoid(x)
    return x

net = Net(input_dim=8, hidden_dim=16, output_dim=1)
print(net)

Net(
  (fc1): Linear(in_features=8, out_features=16, bias=True)
  (batch_norm1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (fc2): Linear(in_features=16, out_features=16, bias=True)
  (layer_norm2): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=16, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
